## Lesson 9 - Ensemble
- 이번 실습 자료에서는 강의시간에 다루었던 Straified kFold Cross Validation(교차 검증)과 이를 활용한 Out-Of-Fold Ensemble 및 Test Time Augmentation에 대해 다뤄보겠습니다. 
    - k개의 Fold로 나누어 교차 검증하는 kFold Cross Validation은 기존 train, valid 로 나누어 한번만 검증하던 프로세스에서 조금 더 엄격하게 모델을 검증하는 방법입니다. 
    - kFold Cross Validation에서 학습한 k개의 모델의 결과를 앙상블 하는 것이 Out-Of-Fold(OOF) Ensemble 입니다.
    - Test Time Augmentation은 Test 데이터의 결과를 예측할 때 Augmentation 기법을 적용해 여러번 예측하고, 예측한 결과들의 평균으로 결과를 내는 것을 말합니다. 
- Sklearn의 kFold는 데이터셋의 인덱스로 Fold를 정의하므로 기존에 사용하던 DataSet이 아닌 인덱스로 생성한 Subset을 사용합니다. 이번 실습 자료에서는 매 이터레이션마다 생성한 SubSet 객체를 활용해 DataLoader를 반환해 학습 및 검증에 사용하게 됩니다.

In [ ]:
import random
import os, sys

import numpy as np
from torch.utils.data import Subset
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import StratifiedKFold

sys.path.append(os.path.abspath('..'))

# BaseLine 코드로 주어진 dataset.py model.py, loss.py를 Import 합니다.
from dataset import MaskBaseDataset, BaseAugmentation
from model import *
from loss import create_criterion

sys.path.append('../')

def seed_everything(seed):
    """
    동일한 조건으로 학습을 할 때, 동일한 결과를 얻기 위해 seed를 고정시킵니다.
    
    Args:
        seed: seed 정수값
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(42)

### Model Parameter Setting

In [ ]:
# -- parameters
# img_root = '학습 이미지 폴더의 경로를 입력해주세요.'

batch_size = 64
num_workers = 4
num_classes = 18

num_epochs = 100  # 학습할 epoch의 수
lr = 1e-4
lr_decay_step = 10
criterion_name = 'cross_entropy' # loss의 이름

train_log_interval = 20  # logging할 iteration의 주기
name = "02_model_results"  # 결과를 저장하는 폴더의 이름

# -- settings
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

### DataLoader
- index를 사용한 Dataloader 정의
- getDataloader 함수 설명
    1. Pytorch Dataset, train 인덱스, valid 인덱스, batch size를 전달받아 Train, Valid DataLoader 객체를 반환합니다.
    2. torch.utils.data.Subset 객체는 데이터셋과 해당 데이터셋의 인덱스를 전달받아 Subset 객체를 생성합니다. 생성한 Subset 객체를 사용해 DataLoader 객체를 반환합니다.

In [ ]:
def getDataloader(dataset, train_idx, valid_idx, batch_size, num_workers):
    # 인자로 전달받은 dataset에서 train_idx에 해당하는 Subset 추출
    train_set = torch.utils.data.Subset(dataset,
                                        indices=train_idx)
    # 인자로 전달받은 dataset에서 valid_idx에 해당하는 Subset 추출
    val_set   = torch.utils.data.Subset(dataset,
                                        indices=valid_idx)
    
    # 추출된 Train Subset으로 DataLoader 생성
    train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last=True,
        shuffle=True
    )
    # 추출된 Valid Subset으로 DataLoader 생성
    val_loader = torch.utils.data.DataLoader(
        val_set,
        batch_size=batch_size,
        num_workers=num_workers,
        drop_last=True,
        shuffle=False
    )
    
    # 생성한 DataLoader 반환
    return train_loader, val_loader

### Stratified k-Fold
1. k를 나타내는 n_splits 값을 설정해 StratifiedKFold 객체를 준비합니다. 
2. skf.split(x, y) 메소드를 사용해 데이터셋의 인덱스를 얻어내고, 라벨(y)을 기준으로 Stratified를 진행합니다. 
3. 매 이터레이션마다 반환받은 인덱스를 사용해 getDataloader 함수에 전달하여 DataLoader를 생성합니다.
4. 나머지 학습 프로세스는 이전 강의에서 사용한 프로세스와 동일하게 진행합니다.
- Stratify는 데이터셋을 분리하기 이전에 ```클래스 비율```을 분리한 이후에도 유지해주는 기능을 말합니다. 

In [ ]:
dataset = MaskBaseDataset(img_root)

transform = BaseAugmentation(
    resize=[128, 96],
    mean=dataset.mean,
    std=dataset.std,
)

dataset.set_transform(transform)

In [ ]:
os.makedirs(os.path.join(os.getcwd(), 'results', name), exist_ok=True)

# 5-fold Stratified KFold 5개의 fold를 형성하고 5번 Cross Validation을 진행합니다.
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)

counter = 0
patience = 10
accumulation_steps = 2
best_val_acc = 0
best_val_loss = np.inf

labels = [dataset.encode_multi_class(mask, gender, age) for mask, gender, age in zip(dataset.mask_labels, dataset.gender_labels, dataset.age_labels)]

# Stratified KFold를 사용해 Train, Valid fold의 Index를 생성합니다.
# labels 변수에 담긴 클래스를 기준으로 Stratify를 진행합니다. 
for i, (train_idx, valid_idx) in enumerate(skf.split(dataset.image_paths, labels)):
    
    # 생성한 Train, Valid Index를 getDataloader 함수에 전달해 train/valid DataLoader를 생성합니다.
    # 생성한 train, valid DataLoader로 이전과 같이 모델 학습을 진행합니다. 
    train_loader, val_loader = getDataloader(dataset, train_idx, valid_idx, batch_size, num_workers)

    # -- model
    model = BaseModel(num_classes=num_classes).to(device)

    # -- loss & metric
    criterion = create_criterion(criterion_name)
    train_params = [{'params': getattr(model.net, 'features').parameters(), 'lr': lr / 10, 'weight_decay':5e-4},
                    {'params': getattr(model.net, 'classifier').parameters(), 'lr': lr, 'weight_decay':5e-4}]
    optimizer = Adam(train_params)
    scheduler = StepLR(optimizer, lr_decay_step, gamma=0.5)

    # -- logging
    logger = SummaryWriter(log_dir=f"results/cv{i}_{name}")
    for epoch in range(num_epochs):
        # train loop
        model.train()
        loss_value = 0
        matches = 0
        for idx, train_batch in enumerate(train_loader):
            inputs, labels = train_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            loss = criterion(outs, labels)

            loss.backward()
            
             # -- Gradient Accumulation
            if (idx+1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            loss_value += loss.item()
            matches += (preds == labels).sum().item()
            if (idx + 1) % train_log_interval == 0:
                train_loss = loss_value / train_log_interval
                train_acc = matches / batch_size / train_log_interval
                current_lr = scheduler.get_last_lr()
                print(
                    f"Epoch[{epoch}/{num_epochs}]({idx + 1}/{len(train_loader)}) || "
                    f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
                )

                loss_value = 0
                matches = 0

        scheduler.step()

        # val loop
        with torch.no_grad():
            print("Calculating validation results...")
            model.eval()
            val_loss_items = []
            val_acc_items = []
            for val_batch in val_loader:
                inputs, labels = val_batch
                inputs = inputs.to(device)
                labels = labels.to(device)

                outs = model(inputs)
                preds = torch.argmax(outs, dim=-1)

                loss_item = criterion(outs, labels).item()
                acc_item = (labels == preds).sum().item()
                val_loss_items.append(loss_item)
                val_acc_items.append(acc_item)

            val_loss = np.sum(val_loss_items) / len(val_loader)
            val_acc = np.sum(val_acc_items) / len(valid_idx)

            # Callback1: validation accuracy가 향상될수록 모델을 저장합니다.
            if val_loss < best_val_loss:
                best_val_loss = val_loss
            if val_acc > best_val_acc:
                print("New best model for val accuracy! saving the model..")
                torch.save(model.state_dict(), f"results/{name}/{epoch:03}_accuracy_{val_acc:4.2%}.ckpt")
                best_val_acc = val_acc
                counter = 0
            else:
                counter += 1
            # Callback2: patience 횟수 동안 성능 향상이 없을 경우 학습을 종료시킵니다.
            if counter > patience:
                print("Early Stopping...")
                break


            print(
                f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
                f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
            )

### Out-Of-Fold Ensemble with TTA
1. 이전 단계에서 작성한 k-Fold Cross Validation 코드에 각 폴드들의 예측 결과를 담을 oof_pred 변수를 준비합니다.
    - 생성한 행렬의 차원은 (Test set의 샘플 수, 예측할 클래스의 개수)로 정의됩니다. 
    - 각 열에는 주어진 이미지가 해당 클래스일 확률을 담습니다. (Soft Voting), 여기에서 확률이 아닌 라벨로 앙상블을 진행하는 경우를 Hard Voting이라 합니다.
    - 확률을 저장할 때는 모든 확률의 합은 1이라는 확률의 정의를 유지하기 위해 확률을 k로 나누어 평균을 취해줍니다.
2. 각 행에 대해 가장 높은 확률을 갖는 클래스를 선택해 결과를 도출합니다.
3. 테스트 셋을 예측할 때 Augmentation 기법을 적용하여 Test Time Augmentation을 진행합니다. 
    - 해당 실습 코드에서는 flip을 적용하여 2번 TTA를 진행합니다.

In [ ]:
import pandas as pd
from torchvision.transforms import Resize, ToTensor, Normalize

test_img_root = '학습 이미지 폴더의 경로' 
# public, private 테스트셋이 존재하니 각각의 예측결과를 저장합니다.

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_img_root, 'info.csv'))
image_dir = os.path.join(test_img_root, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((128, 96), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)),
])
test_dataset = TestDataset(test_img_root, transform)

test_loader = DataLoader(
    test_dataset,
    shuffle=False
)

In [ ]:
os.makedirs(os.path.join(os.getcwd(), 'results', name), exist_ok=True)

n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)

counter = 0
patience = 10
accumulation_steps = 2
best_val_acc = 0
best_val_loss = np.inf
oof_pred = None

labels = [dataset.encode_multi_class(mask, gender, age) for mask, gender, age in zip(dataset.mask_labels, dataset.gender_labels, dataset.age_labels)]

# K-Fold Cross Validation과 동일하게 Train, Valid Index를 생성합니다. 
for i, (train_idx, valid_idx) in enumerate(skf.split(dataset.image_paths, labels)):
    train_loader, val_loader = getDataloader(dataset, train_idx, valid_idx, batch_size, num_workers)

    # -- model
    model = BaseModel(num_classes=num_classes).to(device)

    # -- loss & metric
    criterion = create_criterion(criterion_name)
    train_params = [{'params': getattr(model.net, 'features').parameters(), 'lr': lr / 10, 'weight_decay':5e-4},
                    {'params': getattr(model.net, 'classifier').parameters(), 'lr': lr, 'weight_decay':5e-4}]
    optimizer = Adam(train_params)
    scheduler = StepLR(optimizer, lr_decay_step, gamma=0.5)

    # -- logging
    logger = SummaryWriter(log_dir=f"results/cv{i}_{name}")
    for epoch in range(num_epochs):
        # train loop
        model.train()
        loss_value = 0
        matches = 0
        for idx, train_batch in enumerate(train_loader):
            inputs, labels = train_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            loss = criterion(outs, labels)

            loss.backward()
            
             # -- Gradient Accumulation
            if (idx+1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            loss_value += loss.item()
            matches += (preds == labels).sum().item()
            if (idx + 1) % train_log_interval == 0:
                train_loss = loss_value / train_log_interval
                train_acc = matches / batch_size / train_log_interval
                current_lr = scheduler.get_last_lr()
                print(
                    f"Epoch[{epoch}/{num_epochs}]({idx + 1}/{len(train_loader)}) || "
                    f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}"
                )

                loss_value = 0
                matches = 0

        scheduler.step()

        # val loop
        with torch.no_grad():
            print("Calculating validation results...")
            model.eval()
            val_loss_items = []
            val_acc_items = []
            for val_batch in val_loader:
                inputs, labels = val_batch
                inputs = inputs.to(device)
                labels = labels.to(device)

                outs = model(inputs)
                preds = torch.argmax(outs, dim=-1)

                loss_item = criterion(outs, labels).item()
                acc_item = (labels == preds).sum().item()
                val_loss_items.append(loss_item)
                val_acc_items.append(acc_item)

            val_loss = np.sum(val_loss_items) / len(val_loader)
            val_acc = np.sum(val_acc_items) / len(valid_idx)

            # Callback1: validation accuracy가 향상될수록 모델을 저장합니다.
            if val_loss < best_val_loss:
                best_val_loss = val_loss
            if val_acc > best_val_acc:
                print("New best model for val accuracy! saving the model..")
                torch.save(model.state_dict(), f"results/{name}/{epoch:03}_accuracy_{val_acc:4.2%}.ckpt")
                best_val_acc = val_acc
                counter = 0
            else:
                counter += 1
            # Callback2: patience 횟수 동안 성능 향상이 없을 경우 학습을 종료시킵니다.
            if counter > patience:
                print("Early Stopping...")
                break

            print(
                f"[Val] acc : {val_acc:4.2%}, loss: {val_loss:4.2} || "
                f"best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}"
            )
            
    # 각 fold에서 생성된 모델을 사용해 Test 데이터를 예측합니다. 
    all_predictions = []
    with torch.no_grad():
        for images in test_loader:
            images = images.to(device)

            # Test Time Augmentation
            pred = model(images) / 2 # 원본 이미지를 예측하고
            pred += model(torch.flip(images)) / 2 # flip으로 뒤집어 예측합니다. 
            all_predictions.extend(pred.cpu().numpy())

        fold_pred = np.array(all_predictions)

    # 확률 값으로 앙상블을 진행하기 때문에 'k'개로 나누어줍니다.
    if oof_pred is None:
        oof_pred = fold_pred / n_splits
    else:
        oof_pred += fold_pred / n_splits


### Submission 파일 만들기
- 각 fold에서 예측한 결과의 확률을 평균낸 oof_pred에서 가장 높은 확률을 갖는 클래스를 추출하고 csv 파일을 생성합니다. 

In [ ]:
submission['ans'] = np.argmax(oof_pred, axis=1)
submission.to_csv(os.path.join(test_img_root, 'submission.csv'), index=False)
print('test inference is done!')